# Imports

In [1]:
import torch as th
import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import pandas as pd
import os

from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, GPUStatsMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.utils.class_weight import compute_class_weight
import torchvision.transforms as transforms

# import data module
from dataset import CassavaDM

# import model 
from model import Model

# Config

In [2]:
from config import Config

In [3]:
Config.__dict__

mappingproxy({'__module__': 'config',
              'wording_dir': '/home/zeusdric/Dric/kaggle/cassava-leaf',
              'data_dir': '/home/zeusdric/Dric/kaggle/cassava-leaf/data',
              'train_images_dir': '/home/zeusdric/Dric/kaggle/cassava-leaf/data/train_images',
              'test_images_dir': '/home/zeusdric/Dric/kaggle/cassava-leaf/data/test_images',
              'train_batch_size': 32,
              'test_batch_size': 16,
              'lr': 0.0001,
              'num_epochs': 30,
              'n_folds': 5,
              'models_dir': '/home/zeusdric/Dric/kaggle/cassava-leaf/models',
              'logs_dir': '/home/zeusdric/Dric/kaggle/cassava-leaf/working/logs',
              'submissions_dir': '/home/zeusdric/Dric/kaggle/cassava-leaf/submissions',
              'device': 'cuda',
              'img_size': 300,
              'n_channels': 3,
              'base_model': 'resnet50',
              'freeze': False,
              'precision': 16,
              'num_cl

In [4]:
train_df = pd.read_csv(os.path.join(Config.data_dir, 'train.csv'))
ss = pd.read_csv(os.path.join(Config.submissions_dir, 'sample_submission.csv'))

In [5]:
class_w = compute_class_weight(classes=train_df.label.unique(), y=train_df.label.values, class_weight='balanced')
class_w

array([3.93689052, 0.3252318 , 1.9549566 , 1.79354568, 1.66061312])

# Modeling
* Encoder/Features extractor: esnet, efficientnet, densenet)
* Decoder/Classifier : inear layer (in_tures, n_classes)
* loss_fn : CrossEntropyLoss
* Optimize metrics : accuracy
* Optimizer : Adam, AdamW, SGD
* learning rate : (3e-5...1e-1)
* lr scheduler : linear with warmup, ReduceLROnPlateau
* pretrained : Always true


In [6]:
# ---------------
# data transforms
# ---------------
data_transforms = {
    'train':th.nn.Sequential(
            transforms.Resize((Config.img_size, Config.img_size)),
            transforms.RandomHorizontalFlip(p=.6),
            transforms.RandomVerticalFlip(p=.3),
            transforms.GaussianBlur(3),
        ),
    'test':th.nn.Sequential(
            transforms.Resize((Config.img_size, Config.img_size)),
            transforms.RandomVerticalFlip(p=.3),

        )
}

#------------
# data module
# ----------
dm = CassavaDM(
    df = train_df, 
    frac=1,
    n_classes=Config.num_classes, 
    data_transforms=data_transforms, 
    train_data_dir=Config.train_images_dir,
    test_data_dir=Config.test_images_dir)
dm.setup()
#-------
# Model 
# -----
model = Model(config=Config, 
              len_train_ds=len(dm.train_ds),
              class_w=th.from_numpy(class_w))

[INFO] Training on 19257 samples belonging to 5 classes
[INFO] Validating on 2140 samples belonging to 5 classes


In [7]:
model

Model(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [8]:
model.predict(dm.val_ds[0]['images'])

array([ 0.7973824 ,  0.2894341 ,  0.8548755 , -0.15265591, -0.31367007],
      dtype=float32)

# Training loop

In [ ]:
ckpt_saver = ModelCheckpoint(filename=f'{Config.base_model}-val_acc'
)

es = EarlyStopping(
)
tb_logger = TensorBoardLogger(
)
Callbacks = []

train = pl.Trainer(gpus=-1, 
                   callbacks=Callbacks, 
                   logger=tb_logger,
                   max_epochs=Config.num_epochs)

# Export trained model(s)